Resume NER Part 4: Working with Flair NLP

---

In this part we will use flair NLP to train a model on our data and evaluate the results. Please make sure you have set up your Google account and uploaded your files to Google drive. This Notebook should run on Google Colab.

Let's change the working directory to the Google drive where our training data is, and install flair nlp. 

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
import os
os.chdir("/content/gdrive/My Drive/saki_ex2") 
print(os.getcwd())

/content/gdrive/My Drive/saki_ex2


In [3]:
# download flair library #
! pip install flair

     |████████████████████████████████| 143kB 2.7MB/s 
     |████████████████████████████████| 133kB 32.0MB/s 
     |████████████████████████████████| 655kB 36.1MB/s 
     |████████████████████████████████| 798kB 27.5MB/s 
     |████████████████████████████████| 1.0MB 37.1MB/s 
  Stored in directory: /root/.cache/pip/wheels/bd/57/d3/907c3ee02d35e66f674ad0106e61f06eeeb98f6ee66a6cc3fe
  Stored in directory: /root/.cache/pip/wheels/35/e4/80/abf3b33ba89cf65cd262af8a22a5a999cc28fbfabea6b38473
  Stored in directory: /root/.cache/pip/wheels/15/ee/a8/6112173f1386d33eebedb3f73429cfa41a4c3084556bcee254
  Stored in directory: /root/.cache/pip/wheels/c0/47/fb/8a64f89aecfe0059830479308ad42d62e898a3e3cefdf6ba28
Successfully built sqlitedict regex segtok mpld3


In the next section, we will train a NER model with flair. This code is taken from the flair nlp tutorials section 7. "Training a model" 
https://github.com/zalandoresearch/flair/blob/master/resources/docs/TUTORIAL_7_TRAINING_A_MODEL.md



In [0]:
# imports 
from flair.data import Corpus
from flair.data_fetcher import NLPTaskDataFetcher, NLPTask
from typing import List

# columns of "gold standard" ner annotations and text "text","ner","doc","ner_spacy" doc ner ner_spacy text
columns = {0: "doc", 1: "ner", 2: "ner_spacy", 3: "text"}
#columns = {0: "text", 1: "ner", 2: "ner_spacy", 3: "doc"}
# folder where training and test data are
data_folder = '/content/gdrive/My Drive/saki_ex2/data/'
train_file = 'train3_res_bilou_preprocessed.txt'
test_file = 'test3_res_bilou_preprocessed.txt'

# 2. what tag do we want to predict?
tag_type = 'ner'


In [15]:
downsample = 1.0 # 1.0 is full data, try a much smaller number like 0.01 to test run the code train_res_bilou_preprocessed.txt test_res_bilou_preprocessed.txt
# 1. get the corpus
corpus: Corpus = NLPTaskDataFetcher.load_column_corpus(data_folder, columns, train_file=train_file, test_file=test_file, dev_file=None).downsample(downsample)
print(corpus)

# 3. make the tag dictionary from the corpus
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)
print(tag_dictionary.idx2item)


2019-06-18 14:46:39,637 Reading data from /content/gdrive/My Drive/saki_ex2/data
2019-06-18 14:46:39,644 Train: /content/gdrive/My Drive/saki_ex2/data/train3_res_bilou_preprocessed.txt
2019-06-18 14:46:39,648 Dev: None
2019-06-18 14:46:39,652 Test: /content/gdrive/My Drive/saki_ex2/data/test3_res_bilou_preprocessed.txt


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated function (or staticmethod) load_column_corpus. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/flair/data_fetcher.py:312: DeprecationWarning: Call to deprecated function (or staticmethod) read_column_data. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  train_file, column_format
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n =

Corpus: 9665 train + 1074 dev + 3681 test sentences
[b'<unk>', b'O', b'-', b'B-Designation', b'L-Designation', b'I-Degree', b'L-Degree', b'"B-College', b'"L-College', b'I-Designation', b'B-Degree', b'U-Designation', b'U-Degree', b'"I-College', b'"U-College', b'ner', b'<START>', b'<STOP>']


In [16]:

# 4. initialize embeddings. Experiment with different embedding types to see what gets the best results
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings,FlairEmbeddings, CharacterEmbeddings
from typing import List
embedding_types: List[TokenEmbeddings] = [
    WordEmbeddings('glove'),
    # comment in this line to use character embeddings
    #CharacterEmbeddings(),

    # comment in these lines to use flair embeddings (needs a LONG time to train :-)
    FlairEmbeddings('news-forward'),
    FlairEmbeddings('news-backward'),
]

embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)

# 5. initialize sequence tagger
from flair.models import SequenceTagger

tagger: SequenceTagger = SequenceTagger(hidden_size=256,
                                        embeddings=embeddings,
                                        tag_dictionary=tag_dictionary,
                                        tag_type=tag_type,
                                        use_crf=True)




/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
# 6. initialize trainer
from flair.trainers import ModelTrainer

trainer: ModelTrainer = ModelTrainer(tagger, corpus)

# 7. start training
trainer.train('resources/taggers/resume-ner',
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=80)

# 8. plot training curves (optional)
#from flair.visual.training_curves import Plotter
#plotter = Plotter()
#plotter.plot_training_curves('resources/taggers/example-ner/loss.tsv')
#plotter.plot_weights('resources/taggers/example-ner/weights.txt')


2019-06-18 14:47:00,914 ----------------------------------------------------------------------------------------------------
2019-06-18 14:47:00,917 Evaluation method: MICRO_F1_SCORE
2019-06-18 14:47:01,565 ----------------------------------------------------------------------------------------------------
2019-06-18 14:47:06,463 epoch 1 - iter 0/303 - loss 73.80541992
2019-06-18 14:48:28,843 epoch 1 - iter 30/303 - loss 10.70903275
2019-06-18 14:49:49,133 epoch 1 - iter 60/303 - loss 7.20665994
2019-06-18 14:50:53,323 epoch 1 - iter 90/303 - loss 5.74100014
2019-06-18 14:52:02,913 epoch 1 - iter 120/303 - loss 4.98819512
2019-06-18 14:53:08,309 epoch 1 - iter 150/303 - loss 4.44180673
2019-06-18 14:54:15,098 epoch 1 - iter 180/303 - loss 4.07084383
2019-06-18 14:55:20,531 epoch 1 - iter 210/303 - loss 3.75093430
2019-06-18 14:56:26,924 epoch 1 - iter 240/303 - loss 3.48717626
2019-06-18 14:57:35,521 epoch 1 - iter 270/303 - loss 3.28823881
2019-06-18 14:58:48,643 epoch 1 - iter 300/30